In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import lxml
import tkinter as tk
from tkinter import filedialog

In [2]:
# path = 'XXEJE_EAM___Ciclo_de_vida_órde_100821.xls'
root = tk.Tk()
root.withdraw()

path = filedialog.askopenfilename()
with open(path, 'rt', encoding='utf-8') as myfile:
    data = myfile.read()#.replace('</span><br/><span class="c62">', ' ').replace('</span><br/><br/><span class="c62">', ' ').replace('</span><br/><br/><br/><span class="c62">', ' ')

In [3]:
df_ori = pd.read_html(data, header=0)
# df_cvo = df_ori.copy()
df_cvo = pd.DataFrame(df_ori[0])

In [4]:
# df_cvo = pd.DataFrame(df_ori[0])

In [5]:
# df_cvo.info()

In [6]:
df_cvo = pd.DataFrame(df_ori[0])
# df_cvo.loc[df_cvo['Descripción'].map(type) == datetime, 'Descripción'] = None
# df_cvo['Descripción'].replace('MRB', None, inplace=True)

df_cvo.dropna(how='all', inplace=True)

df_cvo.loc[df_cvo['Descripcion Activo'].isnull(), 'Nulo'] = 'Si'

vacios = df_cvo[df_cvo['Nulo']=='Si'].index.values

# df_cvo.at[i, 'Descripción'] = [df_cvo.loc[i+1, ['Numero Activo','Descripción']]['Descripción'] for i in vacios]

for i in vacios:
    # print(df_cvo.loc[i, ['Numero Activo','Descripción']]['Descripción'])
    df_cvo.at[i, 'Descripción'] = df_cvo.loc[i+1, ['Numero Activo','Descripción']]['Descripción']

df_cvo.drop(columns='Nulo', inplace=True)
df_cvo.drop(index=vacios, inplace=True)
df_cvo.reset_index(drop=True, inplace=True)
df_cvo.drop_duplicates(inplace = True)

df_cvo.insert(loc=2, column='PPA_MMU', value=None)
df_cvo.insert(loc=5, column='Tipo Ot', value='Principal')
df_cvo.insert(loc=6, column='Sistema', value=None)

sist = ['Grupo Motopropulsor', 'Alimentacion De Combustible', 'Frenos', 'Rodadura', 'Direccion y Conduccion', 'Suspencion', 'Generacion e Iluminacion', 'Indicacion y Registro', 'Anfibio y Vadeo', 'Guiado de Armamento', 'Observacion y Control de Fuego', 'Armamento Principal', 'No Identificado 13', 'Plataforma Hidraulica', 'Plataforma Estructural', 'Equipamento Zapador / Recuperador', 'Extincion de Incendios', 'Acondicionamiento de Aire', 'Lanza Fumigenos', 'Estructura', 'Equipamiento Estructura', 'No Identificado 22', 'Comunicaciones']

for i, sis in enumerate(sist):
    # print(str(i+1).zfill(2))
    df_cvo['Sistema'] = np.where(((df_cvo['Grupo De Activos']=='GR_SIS.02.0'+str(i+1).zfill(2)) | (df_cvo['Grupo De Activos'].str.contains('GR_SUB.02.0'+str(i+1).zfill(2)))), sis, df_cvo['Sistema'])
    
df_cvo['Numero Ot Principal'] = np.where(df_cvo['Numero Ot Principal'].str.contains('No tiene'),None,df_cvo['Numero Ot Principal'])
df_cvo['Tipo Ot'] = np.where(df_cvo['Numero Ot Principal'].isnull(),df_cvo['Tipo Ot'],'Secundaria')

In [7]:
df_cvo['PPA_MMU'] = df_cvo['Descripcion Activo'].str.split('_', expand = True)[0]
b=0
for i, n in enumerate(df_cvo['PPA_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            # b=2
            # print(df_cvo_cvo.loc[i,'Numero Activo'][4:])
            m = df_cvo.loc[i,'Numero Activo'][4:]
            df_cvo.loc[i,'PPA_MMU'] = m # df_cvo.loc[i,'Numero Activo'][4:]
             
        # else:
            # print(n)
    except:
        b=1

for i, n in enumerate(df_cvo['PPA_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            # b=2
            # print(df_cvo_cvo.loc[i,'Descripción Activo'][:7])
            df_cvo.loc[i,'PPA_MMU'] = df_cvo.loc[i,'Descripcion Activo'][:7]
        # else:
            # print(n)
    except:
        b=1

for i, n in enumerate(df_cvo['PPA_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            # print(df_cvo.loc[i,'Descripción Activo'][:7])
            # print(n)
            df_cvo.loc[i,'PPA_MMU'] = 'SinNumero'
        else:
            b=2
    except:
        b=1

NameError: name 'df_' is not defined

In [ ]:
# df_cvo['PPA_MMU'][df_cvo['PPA_MMU'].isna()] = 'SinNumero'

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_2216/2337654648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cvo['PPA_MMU'][df_cvo['PPA_MMU'].isna()] = 'SinNumero'


In [9]:
_, df_rs = pd.read_html('XXEJE_EAM____Reporte_situación_021121.xls', header =0)
df_bdvehic = pd.read_csv('Clean_Bd_Vehic.csv', encoding='latin-1', index_col=0)
df_cod_org = pd.read_csv('Codigo_Organizacion.csv', sep=';')

In [10]:
df_rs.columns = df_rs.columns.str.replace(' ', '')
df_cvo.columns = df_cvo.columns.str.replace(' ', '')

df_rs.columns = df_rs.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_cvo.columns = df_cvo.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [11]:
column_names = ['Numero_OT', 'OT_Principal', 'Tipo_OT', 'Estado_OT', 'PPA_MMU', 'SIC', 'Estado_SIC', 'Total', 'Desc', 'Marca', 'Modelo', 'Nom', 'Ele_Comp', 'Desc_Falla', 'Cod_Org', 'Nom_Org', 'Cod_Taller', 'UAC', 'UR', 'Tipo_Mant', 'Tipo_Comb', 'Sistema', 'Cod_Falla', 'Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre', 'Year']

df = pd.DataFrame(columns = column_names)
df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.columns = df.columns.str.replace(' ', '')

In [12]:
df_cvo.columns

Index(['UAC', 'UR', 'PPA_MMU', 'NombreUR', 'NumeroActivo', 'TipoOt', 'Sistema',
       'Solicitud', 'Fechasolicitud', 'NumeroOt', 'NumeroOtPrincipal',
       'DescripcionOt', 'FechaCreacion', 'FechaLiberacion',
       'FechaFinalizacion', 'FechaCierre', 'Estado', 'MotivoEstado',
       'DescripcionActivo', 'GrupoDeActivos', 'TipoMantenimiento',
       'CodigoTaller', 'CodigoSolucion', 'DescripcionFalla',
       'DescripcionCausa', 'DescripcionSolucion', 'ComentarioFalla',
       'FechaFalla'],
      dtype='object')

In [13]:
df[['Numero_OT', 'OT_Principal', 'Tipo_OT', 'Estado_OT', 'PPA_MMU', 'Ele_Comp', 'Desc_Falla', 'Cod_Org', 'Cod_Taller', 'Tipo_Mant', 'Sistema', 'Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre']] = df_cvo[['NumeroOt', 'NumeroOtPrincipal', 'TipoOt', 'Estado', 'PPA_MMU', 'GrupoDeActivos', 'DescripcionOt', 'NombreUR', 'CodigoTaller', 'TipoMantenimiento', 'Sistema', 'FechaCreacion', 'FechaFinalizacion', 'FechaLiberacion', 'FechaCierre']]
# df['OT_Principal'] = df_cvo['NumeroOtPrincipal']
# df['Tipo_OT'] = df_cvo['TipoOt']
# df['Estado_OT'] = df_cvo['Estado']
# # df['Prioridad_Ot'] = df_cvo['Prioridad']
# df['PPA_MMU'] = df_cvo['PPA_MMU']
# df['Ele_Comp'] = df_cvo['GrupoDeActivos']
# df['Desc_Falla'] = df_cvo['Descripcion']
# df['Cod_Org'] = df_cvo['CodOrganizacion']
# df['Cod_Taller'] = df_cvo['CodigoTaller']
# df['Tipo_Mant'] = df_cvo['TipoMantenimiento']
# df['Sistema'] = df_cvo['Sistema']
# df['Cod_Falla'] = df_cvo['CodigoFalla']
# df['Fec_Creacion'] = df_cvo['FechaCreacion']
# df['Fec_Inicial_Prog'] = df_cvo['FechaInicialProgramada']
# df['Fec_Final_Prog'] = df_cvo['FechaFinalizacionProgramada']
# df['Fec_Final'] = df_cvo['FechaFinalizacion']
# df['Fec_Liberacion'] = df_cvo['FechaLiberacion']
# df['Fec_Cierre'] = df_cvo['FechaCierre']
# df['Fec_Falla'] = df_cvo['FechaFalla']
# df['Hora_Rep'] = df_cvo['Duracion']

In [14]:
df['Tipo_OT'] = np.where(df['OT_Principal'].str.contains('No Tiene'), 'Principal', df['Tipo_OT'])
df['Tipo_OT'] = np.where(df['OT_Principal'].str.contains('OT'), 'Secundaria', df['Tipo_OT'])

mp = {'MPI', 'MPC'}
df.loc[df.Tipo_Mant.isin(mp), 'Tipo_OT'] = 'Principal'

In [15]:
def right_join_test(left_db, right_db, left_col, left_merge_cols, right_col, right_merge_cols):
    left_temp_df = left_db.copy()
    right_temp_df = right_db[right_col]
    right_db[right_merge_cols] = pd.merge(left_temp_df, right_temp_df, left_on=left_col, right_on=right_col, how='right', indicator=True)[left_merge_cols]

In [16]:
left_cols = ['Descripcion', 'ATT8', 'Modelo', 'UAC', 'UR', 'ATT11', 'ATT19', 'ATT24']
right_cols = ['Desc', 'Marca', 'Modelo', 'UAC', 'UR', 'Nom', 'Tipo_Comb', 'Ele_Comp']
right_join_test(df_bdvehic, df, 'Numero Etiqueta', left_cols, 'PPA_MMU', right_cols)
df['Ele_Comp'].replace(np.nan, 'NO APLICA', inplace=True)

In [17]:
# left_temp_df = df_bdvehic.copy()
# right_temp_df = df['PPA_MMU']
# left_cols = ['Descripcion', 'ATT8', 'Modelo', 'UAC', 'UR', 'ATT11', 'ATT19', 'ATT24']
# right_cols = ['Desc', 'Marca', 'Modelo', 'UAC', 'UR', 'Nom', 'Tipo_Comb', 'Ele_Comp']
# df[right_cols] = pd.merge(left_temp_df, right_temp_df, left_on='Numero Etiqueta', right_on='PPA_MMU', how='right', indicator=True)[left_cols]
# df['Ele_Comp'].replace(np.nan, 'NO APLICA', inplace=True)

In [18]:
left_cols = ['NumeroSic', 'EstadoSic', 'Total']
right_cols = ['SIC', 'Estado_SIC', 'Total']
right_join_test(df_rs, df, 'NroOt', left_cols, 'Numero_OT', right_cols)

In [19]:
# left_temp_df = df_rs.copy()
# right_temp_df = df['Numero_OT']
# df[['SIC', 'Estado_SIC', 'Total']] = pd.merge(left_temp_df, right_temp_df, left_on='NroOt', right_on='Numero_OT', how='right', indicator=True)[['NumeroSic', 'EstadoSic', 'Total']]

In [20]:
# df_cod_org.head()

In [21]:
left_cols = 'Nombre'
right_cols = 'Nom_Org'
right_join_test(df_cod_org, df, 'Codigo', left_cols, 'Cod_Org', right_cols)

In [22]:
# left_temp_df = df_cod_org.copy()
# right_temp_df = df['Cod_Org']
# df['Nom_Org'] = pd.merge(left_temp_df, right_temp_df, left_on='Codigo', right_on='Cod_Org', how='right', indicator=True)['Nombre']

In [23]:
df_temp = df[['Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre']]

for i in df_temp:
    df_temp.loc[:, i] = pd.to_datetime(df_temp.loc[:, i], errors='coerce')

C:\Users\Lanzet\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
C:\Users\Lanzet\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [24]:
# df_temp

In [25]:
df_temp.replace(np.datetime64('NaT'), str(pd.to_datetime('today').normalize().year), inplace=True)

C:\Users\Lanzet\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [26]:
# df_temp.head()

In [27]:
df['Year'] = [np.amin(np.array([row.Fec_Creacion.year,row.Fec_Final.year,row.Fec_Liberacion.year,row.Fec_Cierre.year])) for _, row in df_temp.iterrows()]

In [28]:
# df['Year'].value_counts()

In [29]:
# df['Year'].unique()

In [30]:
# df.info()

In [31]:
out_file = Path.cwd() / "output.xlsx"
writer = pd.ExcelWriter(out_file, engine = 'xlsxwriter')
df.to_excel(writer, index = False, sheet_name = 'Test1')
#df_vehic.to_excel(writer, index = False, sheet_name = 'Test2')
writer.save()